In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install textstat evaluate

In [ ]:
!pip install rouge_score

In [ ]:
import torch
import torch.nn as nn
from datasets import load_dataset, Dataset, load_from_disk
from transformers import AutoTokenizer, AutoModelForCausalLM, get_scheduler, DataCollatorWithPadding, AutoConfig
from tqdm import tqdm
from torch.optim import AdamW
import os
import json
import nltk
import re
import textstat
from datasets import Dataset
from torch.optim import AdamW
from torch.utils.data import DataLoader
import evaluate

In [ ]:
TOKENIZER_NAME = 'HuggingFaceTB/SmolLM2-135M'
model_name = TOKENIZER_NAME

In [ ]:
train_groups = []
val_groups = []
test_groups = []
with open("/kaggle/input/babylm/train_dataset.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        train_groups.append(json.loads(line))

with open("/kaggle/input/babylm/val_dataset.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        val_groups.append(json.loads(line))

with open("/kaggle/input/babylm/test_dataset.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        test_groups.append(json.loads(line))

In [ ]:
nltk.download('punkt')

TOKENIZER_NAME = "HuggingFaceTB/SmolLM2-135M"
BLOCK_SIZE = 1024
BATCH_SIZE = 2

In [ ]:
from tqdm import tqdm
import copy

tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_NAME)
eos_id = tokenizer.eos_token_id
USE_EOS = eos_id is not None

def pack_paragraph_sentences(paragraph_text, tokenizer, block_size, use_eos=True, eos_id=None):
    sents = nltk.tokenize.sent_tokenize(paragraph_text)
    blocks = []
    cur = []

    for s in sents:
        ids = tokenizer(" " + s, add_special_tokens=False).input_ids

        if use_eos and eos_id is not None:
            ids = ids + [eos_id]

        if len(ids) > block_size:
            if cur:
                blocks.append(cur)
                cur = []

            for i in range(0, len(ids), block_size):
                chunk = ids[i : i + block_size]
                blocks.append(chunk)
            continue

        if len(cur) + len(ids) <= block_size:
            cur.extend(ids)
        else:
            blocks.append(cur)
            cur = ids.copy()

    if cur:
        blocks.append(cur)
    return blocks

blocks = []
for grp in tqdm(train_groups, desc="Tokenizing groups"):
    text = grp["text"]
    paras = text.split("\n\n")
    for p in paras:
        if not p.strip():
            continue
        blocks.extend(pack_paragraph_sentences(p, tokenizer, BLOCK_SIZE,
                                              use_eos=USE_EOS, eos_id=eos_id))

blocks = [b for b in blocks if len(b) > 0]
assert all(1 <= len(b) <= BLOCK_SIZE for b in blocks)

dataset_dict = {
    "input_ids": [b.copy() for b in blocks],
    "attention_mask": [[1] * len(b) for b in blocks],
    "labels": [b.copy() for b in blocks],
}

print(f"Total train blocks: {len(blocks)}")
train_dataset = Dataset.from_dict(dataset_dict)
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


In [ ]:
blocks = []
for grp in tqdm(val_groups, desc="Tokenizing groups"):
    text = grp["text"]
    paras = text.split("\n\n")
    for p in paras:
        if not p.strip():
            continue
        blocks.extend(pack_paragraph_sentences(p, tokenizer, BLOCK_SIZE,
                                              use_eos=USE_EOS, eos_id=eos_id))

blocks = [b for b in blocks if len(b) > 0]
assert all(1 <= len(b) <= BLOCK_SIZE for b in blocks)



dataset_dict = {
    "input_ids": blocks,
    "attention_mask": [[1]*len(b) for b in blocks],
    "labels": blocks.copy(),
}

print(f"Total val blocks: {len(blocks)}")

val_dataset = Dataset.from_dict(dataset_dict)
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


blocks = []
for grp in tqdm(test_groups, desc="Tokenizing groups"):
    text = grp["text"]
    paras = text.split("\n\n")
    for p in paras:
        if not p.strip():
            continue
        blocks.extend(pack_paragraph_sentences(p, tokenizer, BLOCK_SIZE,
                                              use_eos=USE_EOS, eos_id=eos_id))

blocks = [b for b in blocks if len(b) > 0]
assert all(1 <= len(b) <= BLOCK_SIZE for b in blocks)



dataset_dict = {
    "input_ids": blocks,
    "attention_mask": [[1]*len(b) for b in blocks],
    "labels": blocks.copy(),
}

print(f"Total test blocks: {len(blocks)}")

test_dataset = Dataset.from_dict(dataset_dict)
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

In [ ]:
train_dataset

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoModelForCausalLM

class FullDiffWrapper(nn.Module):
    def __init__(self, base_model_name: str):

        super().__init__()
        cfg  = AutoConfig.from_pretrained(base_model_name)
        self.base = AutoModelForCausalLM.from_config(cfg)

        for i, block in enumerate(self.base.model.layers):
    
            dtype = block.input_layernorm.weight.dtype
            device = block.input_layernorm.weight.device
            block.alpha = nn.Parameter(torch.tensor(1.0, dtype=dtype, device=device))
            block.beta  = nn.Parameter(torch.tensor(0.0, dtype=dtype, device=device))
            block.gamma = nn.Parameter(torch.tensor(0.0, dtype=dtype, device=device))
    
            orig_forward = block.forward
    
            def make_patched(orig_forward, alpha, beta, gamma):
                def patched_forward(hidden_states, *args, **kwargs):
                    outputs = orig_forward(hidden_states, *args, **kwargs)
                    cur = outputs[0]
        
                    zeros1 = torch.zeros_like(cur[:, :1, :])
                    prev   = torch.cat([zeros1, cur[:, :-1, :]], dim=1)
        
                    zeros2 = torch.zeros_like(cur[:, :2, :])
                    prev2  = torch.cat([zeros2, cur[:, :-2, :]], dim=1)
        
                    vel = cur - prev
        
                    acc = vel - (prev - prev2)
        
                    new_hidden = (
                        alpha * cur
                      + beta  * vel
                      + gamma * acc
                    )
        
                    return (new_hidden, *outputs[1:])
                return patched_forward
            block.forward = make_patched(orig_forward, block.alpha, block.beta, block.gamma)

    def forward(self, input_ids=None, inputs_embeds=None, **kwargs):
        return self.base(input_ids=input_ids,
                         inputs_embeds=inputs_embeds,
                         **kwargs)

In [ ]:
newmodel = FullDiffWrapper('HuggingFaceTB/SmolLM-135M')
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    newmodel.base.resize_token_embeddings(len(tokenizer))

In [ ]:
class DataCollatorWithPaddingAndLabels(DataCollatorWithPadding):
    def __call__(self, features):
        labels = [torch.tensor(f["labels"]) for f in features]

        features_for_pad = [{k: v for k, v in f.items() if k != "labels"} for f in features]

        batch = super().__call__(features_for_pad)

        batch["labels"] = torch.nn.utils.rnn.pad_sequence(
            labels, batch_first=True, padding_value=-100 
        )
        return batch 


data_collator = DataCollatorWithPaddingAndLabels(tokenizer=tokenizer, return_tensors="pt")

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4,        
    pin_memory=True,
    collate_fn=data_collator)
val_loader = DataLoader(
    val_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False,
    num_workers=4,        
    pin_memory=True,      
    collate_fn=data_collator)

In [ ]:
def evaluate_on_val(model, val_loader, device):
    model.eval()
    total_loss = 0
    total_tokens = 0
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validating"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )
            loss = outputs.loss
            active_tokens = (labels != -100).sum().item()
            total_loss += loss.item() * active_tokens
            total_tokens += active_tokens


    avg_loss = total_loss / (total_tokens + 1e-12)
    perplexity = torch.exp(torch.tensor(avg_loss))
    print(f"Validation loss: {avg_loss:.4f}, Perplexity: {perplexity:.4f}")
    return avg_loss, perplexity.item()

In [ ]:
def save_checkpoint(save_dir, model, tokenizer, optimizer, scheduler, scaler, epoch, step, best_val=None, keep=5):
    os.makedirs(save_dir, exist_ok=True)
    os.makedirs(os.path.join(save_dir, "hf_base"), exist_ok=True)
    # Save model + tokenizer only for best/epoch/final (optional)
    model.base.save_pretrained(os.path.join(save_dir, "hf_base"))
    tokenizer.save_pretrained(os.path.join(save_dir, "hf_base"))
    diff = {k:v for k,v in model.state_dict().items() if any(x in k for x in ("alpha","beta","gamma"))}

    ckpt = {
        "diff":diff,
        "optimizer": optimizer.state_dict(),
        "scheduler": scheduler.state_dict() ,
        "scaler": scaler.state_dict(),
        "epoch": epoch,
        "step": step,
        "best_val": best_val,
        "rng": {
            "torch": torch.get_rng_state(),
            "cuda": torch.cuda.get_rng_state_all() if torch.cuda.is_available() else None,
            "numpy": np.random.get_state(),
            "random": random.getstate(),
        }
    }
    fname = f"ckpt_step_{step:07d}.pt"
    tmp_path = os.path.join(save_dir, "tmp_" + fname)
    final_path = os.path.join(save_dir, fname)
    torch.save(ckpt, tmp_path)
    os.replace(tmp_path, final_path)

    print(f"Saved checkpoint {final_path}")

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
seed = 42
import random, numpy as np
torch.manual_seed(seed)
if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)


In [ ]:
# ------------------------
# Model Setup
# ------------------------
from torch.cuda.amp import autocast, GradScaler
EVALUATE_EVERY_N_STEPS = 5000
CHECKPOINT_DIR = "/kaggle/working/my_model_checkpoints"
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

print("Tokenizer vocab size:", len(tokenizer))
print("Embedding weight shape:", newmodel.base.get_input_embeddings().weight.shape)

newmodel.float()
newmodel.to(device)
max_epochs = 10
ACCUMULATION_STEPS = 4
steps_per_epoch = len(train_loader)
total_steps = int(steps_per_epoch * max_epochs)

optimizer = AdamW(newmodel.parameters(), lr=4e-4)
scheduler = get_scheduler(
    name="cosine",
    optimizer=optimizer,
    num_warmup_steps=int(0.1 * total_steps),
    num_training_steps=total_steps
)
scaler = GradScaler()
step = 0
best_val = float('inf')
try:
    optimizer.zero_grad()
    for epoch in range(max_epochs):
        newmodel.train()
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}")
        for batch in pbar:
            # if step > 5002:
            #     break
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            with autocast(dtype=torch.float16):
                outputs = newmodel(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    labels=labels
                )
                loss = outputs.loss / ACCUMULATION_STEPS
            scaler.scale(loss).backward()            
            if (step + 1) % ACCUMULATION_STEPS == 0:
                scaler.step(optimizer)  # Optimizer step with scaled gradients
                scaler.update()         # Update scaler
                scheduler.step()
                optimizer.zero_grad()
            step += 1
            pbar.set_postfix(loss=outputs.loss.item(), lr=scheduler.get_last_lr()[0])

            if step % EVALUATE_EVERY_N_STEPS == 0:
                avg_loss, perplexity = evaluate_on_val(newmodel, val_loader, device)

        if step % ACCUMULATION_STEPS != 0:
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()
            optimizer.zero_grad()
        avg_loss, perplexity = evaluate_on_val(newmodel, val_loader, device)
        if avg_loss < best_val:
            best_val = avg_loss
        save_checkpoint(CHECKPOINT_DIR, newmodel, tokenizer, optimizer, scheduler, scaler, epoch, step, best_val)
        print(f"Saved epoch {epoch + 1}")

finally:
    save_checkpoint(CHECKPOINT_DIR, newmodel, tokenizer, optimizer, scheduler, scaler, epoch, step, best_val)
    print(f"Training complete.")

In [ ]:
print('herere')

In [ ]:
device

In [ ]:
!wget https://files.osf.io/v1/resources/ryjfm/providers/osfstorage/6819f54f5dc6fc2bff0a7bba/?zip=

In [ ]:
!mv /kaggle/working/index.html?zip= /kaggle/working/evaluation_data.zip

In [ ]:
!unzip /kaggle/working/evaluation_data.zip

In [ ]:
mkdir ./evaluation_data

In [ ]:
!mv /kaggle/working/full_eval /kaggle/working/evaluation_data/full_eval
!mv /kaggle/working/fast_eval /kaggle/working/evaluation_data/fast_eval

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=data_collator)

In [ ]:
!git clone https://github.com/babylm/evaluation-pipeline-2025.git

In [ ]:
!PYTHONPATH="/kaggle/working/evaluation-pipeline-2025" \
/kaggle/working/evaluation-pipeline-2025/eval_zero_shot_fast.sh \
/kaggle/working/version_last_layer/final_model \
v4 \
causal

In [ ]:
import math
import torch
from tqdm import tqdm

bleu_metric  = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")

def evaluate_model(model, test_dataloader, device):
    model.eval()
    model.to(device)

    total_loss = 0
    num_tokens  = 0

    generated_texts = []
    reference_texts = []

    pad_id = tokenizer.pad_token_id or tokenizer.eos_token_id

    with torch.no_grad():
        for batch in tqdm(test_dataloader, desc="Evaluating"):
            input_ids      = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels         = batch["labels"].to(device)

            # 1) forward + loss
            outputs = model(input_ids=input_ids,
                            attention_mask=attention_mask,
                            labels=labels)
            loss = outputs.loss
            total_loss += loss.item() * labels.numel()
            num_tokens  += labels.numel()

            # 2) generation
            gen_ids = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=labels.shape[1] + 1
            )
            decoded_preds = tokenizer.batch_decode(
                gen_ids, skip_special_tokens=True)

            # 3) decode labels
            labels_for_decode = labels.clone()
            labels_for_decode[labels_for_decode == -100] = pad_id
            decoded_labels = tokenizer.batch_decode(
                labels_for_decode, skip_special_tokens=True)

            generated_texts.extend(decoded_preds)
            reference_texts.extend(decoded_labels)

    ppl = math.exp(total_loss / num_tokens)
    bleu  = bleu_metric.compute(
        predictions=generated_texts,
        references=[[r] for r in reference_texts]
    )["bleu"]
    rouge = rouge_metric.compute(
        predictions=generated_texts,
        references=reference_texts
    )

    print(f"\nPerplexity: {ppl:.4f}")
    print(f"BLEU:       {bleu:.4f}")
    print(f"ROUGE:      {rouge}")

    return {"perplexity": ppl, "bleu": bleu, "rouge": rouge}

In [ ]:
evaluate_model(newmodel.base, test_loader, device)